In [1]:
from Database import GSDatabase
from uti import timeit, DataLoader, Logger

GSD = GSDatabase()
DL = DataLoader()
logger = Logger()

C:\Users\JayYang\Platform\NBC\Path\settings.py C:\Users\JayYang\Platform\NBC\Path\settings.py
Base path C:/Users/JayYang
Platform path C:/Users/JayYang/Platform
Onedrive directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital
Daily path C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Daily
Database path C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC
Current working directory C:\Users\JayYang\Platform\NBC
09:50:04 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\Log/20220524 created.
09:50:04 Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0
09:50:04 Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0


In [2]:
from Database.report_type import rc_filter, er_filter, io_filter, ec_filter
import numpy as np

In [3]:
senti = DL.loadDB('NBC sentiment.csv')

09:50:05 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\Log/20220524/20220524095004.txt created.
09:50:05 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\NBC sentiment.csv


In [ ]:
er_ind = er_filter(senti['headline'], senti['summary'])
senti.loc[er_ind]

rc_ind = rc_filter(senti['headline'], senti['summary'])
senti.loc[rc_ind]

# senti[senti['rating_prev'] != senti['rating_curr']]

In [21]:
REPORT_TYPE_MAPPING = {'-1': 'ad-hoc',
                       '0': 'Earning\'s Review',
                       '1': 'Rating Change',
                       '2': 'Target Price Change',
                       '3': 'Estimate Change',
                       '4': 'Initiation'}

In [26]:
for report_type in REPORT_TYPE_MAPPING.values():
    senti[report_type] = ''
    

er_mask = er_filter(senti['headline'], senti['summary'])
rc_mask = senti['rating_curr'] != senti['rating_prev']
# Backfill rating_prev with previous rating_curr

tp_mask = senti['tp_curr'] != senti['tp_prev']
io_mask = io_filter(senti['headline'], senti['summary'])
ec_mask = ec_filter(senti['headline'], senti['summary'])

senti['report_type'] = 'ad-hoc'
senti['report_type'] = np.where(er_mask, 'Earning\'s Review', senti['report_type'])
senti['report_type'] = np.where(ec_mask, 'Estimate Change', senti['report_type'])
senti['report_type'] = np.where(io_mask, 'Initiation', senti['report_type'])
senti['report_type'] = np.where(tp_mask, 'Target Price Change', senti['report_type'])
senti['report_type'] = np.where(rc_mask, 'Rating Change', senti['report_type'])

In [27]:
senti['report_type'].value_counts()

Target Price Change    4862
Earning's Review       4590
ad-hoc                 1223
Estimate Change         626
Rating Change           530
Initiation              188
Name: report_type, dtype: int64

In [28]:
DL.toDB(senti, 'NBC sentiment.csv')

In [29]:
REPORT_TYPE_MAPPING.values()

dict_values(["Earning's Review", 'Rating Change', 'Target Price Change', 'Estimate Change', 'Initiation'])